Primero cargaremos todo las librerias que usaremos y tambien el archivo.csv de canciones

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
from sklearn.neighbors import NearestNeighbors

songs= pd.read_csv('../spotify_songs.csv')
songs.head()

,track_id,track_name,track_artist,lyrics,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,language
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,41,1srJQ0njEQgd8w4XSqI4JQ,Trip,2001-01-01,Pinoy Classic Rock,37i9dQZF1DWYDQ8wBxd7xt,...,-10.068,1,0.0236,0.27900,0.01170,0.0887,0.566,97.091,235440,tl
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",28,3z04Lb9Dsilqw68SHt6jLB,Love & Loss,2017-11-21,Hard Rock Workout,3YouF0u7waJnolytf9JCXf,...,-4.739,1,0.0442,0.01170,0.00994,0.3470,0.404,135.225,373512,en
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0,6oZ6brjB8x3GoeSYdwJdPc,Gold,2005-01-01,"Back in the day - R&B, New Jack Swing, Swingbe...",3a9y4eeCJRmG9p4YKfqYIx,...,-7.504,0,0.2160,0.00432,0.00723,0.4890,0.650,111.904,262467,en
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,41,3ssspRe42CXkhPxdc12xcp,CeeLo's Magic Moment,2012-10-29,Christmas Soul,6FZYc2BvF7tColxO8PBShV,...,-5.819,0,0.0341,0.68900,0.00000,0.0664,0.405,118.593,243067,en
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,65,7h5X3xhh3peIK9Y0qI5hbK,KARD 2nd Digital Single ‘Dumb Litty’,2019-09-22,K-Party Dance Mix,37i9dQZF1DX4RDXswvP6Mj,...,-1.993,1,0.0409,0.03700,0.00000,0.1380,0.240,130.018,193160,en


Cargamos el csv de vector caracteristico de las canciones

In [59]:
features = pd.read_csv('../new_features.csv')
features.head()

,track_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,047fCsbO4NdmwCBn8pcUXl,0.564548,0.975257,0.741779,0.778260,0.667811,0.609394,0.833157,0.609372,0.922743,...,0.010307,0.016957,0.162526,0.135950,0.071303,0.086266,0.007575,0.041471,0.206271,1.000000
1,05oFyvdztPdpwSwpoclD4c,0.696825,0.290768,0.841058,0.593984,0.555382,0.727198,0.333626,0.579363,0.603258,...,0.624171,0.937361,0.734321,0.416651,0.287155,0.390715,0.453160,0.300723,0.202658,0.430568
2,03sno6MIkoVX5eC4pRKMzr,0.000000,0.000000,0.000000,0.000000,0.000000,0.077863,0.914214,0.774472,0.720151,...,0.443171,0.237403,0.147007,0.414582,0.313377,0.280867,0.384517,0.445165,0.438335,0.295929
3,05LCfd8s9n2teUcsTsBAnr,0.468313,0.486041,0.207143,0.206137,0.259841,0.298484,0.124511,0.123562,0.082146,...,0.893339,0.775425,0.878335,0.823169,0.821835,0.692960,0.680213,0.582898,0.377783,0.310036
4,044fQJmT2fZUYcA3BnaOL2,0.866249,0.918441,0.864241,0.893900,0.960604,0.985448,0.951181,0.940388,0.861772,...,0.404450,0.181432,0.270506,0.212315,0.333065,0.214189,0.134198,0.136429,0.256356,0.267438


Verficamos que tenemos lo necesario par auna busqueda textual con pandas solo extraemos las columnas "track_name", "track_artist", "lyrics","track_id". Esto no es necesario, pero se implemento con KNN.

In [60]:
essential = songs[["track_name", "track_artist", "lyrics","track_id"]]
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Limpieza de datos nulos y duplicados

In [61]:
essential = essential.dropna()
essential = essential.drop_duplicates()
essential.shape
essential.head()

,track_name,track_artist,lyrics,track_id
0,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0017A6SJgTbfQVU2EtsPNo
1,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",004s3t0ONYlzxII9PLgU6z
2,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",00chLpzhgVjxs1zKC9UScL
3,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,00cqd6ZsSkLZqGMlQCR0Zo
4,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,00emjlCv9azBN0fzuuyLqy


Obtenemos el tfidvectorizer de nuestra dataframe

In [62]:
vectorizer = TfidfVectorizer()
vectorizer
X = vectorizer.fit_transform(essential["lyrics"])
X

<18194x102102 sparse matrix of type '<class 'numpy.float64'>'
	with 2120968 stored elements in Compressed Sparse Row format>

KNN secuencial de busqueda textual implementada

In [63]:
def KNN_secuencial(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = results.argsort()[-k:][::-1]
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs
query = "I'm a rebel just for kicks, now"
k=5
print(KNN_secuencial(query, k))




                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
12584           Rebel Rebel - 1997 Remaster        David Bowie


Tambien se implemento una variacion con heap

In [64]:
import heapq
def KNN_secuencial_heap(query, k):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X,query_vec).reshape((-1,))
    top_k_indices = heapq.nlargest(k, range(len(results)), results.take)
    top_k_songs = essential.iloc[top_k_indices][["track_name", "track_artist"]]
    return top_k_songs

print(KNN_secuencial_heap(query, k))

                                 track_name       track_artist
15794                         Feel It Still  Portugal. The Man
17317                         Feel It Still  Portugal. The Man
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie


Se implemento una busqueda textual con knn_range_search, no es necesario

In [65]:
radius = 0.2
def KNN_range_Search(query, radius):
    query_vec = vectorizer.transform([query])
    results = cosine_similarity(X, query_vec).reshape((-1,))
    indices = np.where(results >= radius)[0]
    songs = essential.iloc[indices][["track_name", "track_artist"]]
    return songs

print(KNN_range_Search(query, radius))

                                 track_name       track_artist
2006    It's All on U - T-Mass & LZRD Remix           ILLENIUM
5462            Rebel Rebel - 2016 Remaster        David Bowie
8709   It's All on U (feat. Liam O'Donnell)           ILLENIUM
9937                              Pumped Up          Klingande
11489                               McQueen          TrifeDrew
12584           Rebel Rebel - 1997 Remaster        David Bowie
14385           Rebel Rebel - 1999 Remaster        David Bowie
15399                             Pumped Up          Klingande
15720           Rebel Rebel - 2014 Remaster        David Bowie
15786              Rebel Rebel - Remastered        David Bowie
15794                         Feel It Still  Portugal. The Man
16985                         Teenage Kicks     The Undertones
17317                         Feel It Still  Portugal. The Man
18306                       Pumped Up Kicks  Foster The People


In [66]:
first_value = songs.loc[0]
print(first_value)

track_id                                               0017A6SJgTbfQVU2EtsPNo
track_name                                                           Pangarap
track_artist                                                  Barbie's Cradle
lyrics                      Minsan pa Nang ako'y napalingon Hindi ko alam ...
track_popularity                                                           41
track_album_id                                         1srJQ0njEQgd8w4XSqI4JQ
track_album_name                                                         Trip
track_album_release_date                                           2001-01-01
playlist_name                                              Pinoy Classic Rock
playlist_id                                            37i9dQZF1DWYDQ8wBxd7xt
playlist_genre                                                           rock
playlist_subgenre                                                classic rock
danceability                                                    

Esta parte es lo que se pidio implementar, extraemos las mismas columnas importantes "track_name", "track_artist", "lyrics","track_id" para verificar si resulto con existe vemos las primeras filas con .head() de nuestra dataframe.

In [67]:
essential_2 = songs[['track_id', 'track_name', 'track_artist', 'lyrics']]
essential_2.head()

,track_id,track_name,track_artist,lyrics
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu..."
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U..."
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...


Limpieza de datos:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**

Verificamos la cantidad de elementos se redujo o se mantuvo

In [68]:
essential_2 = essential_2.dropna()
essential_2 = essential_2.drop_duplicates()
essential_2.shape


(18194, 4)

Hacemos el merge entre nuestra dataframe extraida essential2 y nuestro csv con el vector caracteristico cargado anteriormente.


In [69]:
combined_data = pd.merge(essential_2, features, on='track_id')
combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0.470423,0.732632,0.855973,0.962355,0.603475,0.635956,...,0.038902,0.122038,0.073865,0.021143,0.019284,0.063284,0.059732,0.060316,0.026597,0.048140
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.551716,0.223094,0.291287,0.160346,0.045806,0.151824,0.163924,0.124293,0.098941,0.066165
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.639059,0.712995,0.630074,0.718678,0.465735,0.600425,...,0.564603,0.547191,0.585334,0.650047,0.622460,0.714395,0.713101,0.656200,0.686142,0.817051
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,0.690065,0.800364,0.246730,0.239026,0.128097,0.170915,...,0.105633,0.068960,0.088050,0.215053,0.203416,0.047231,0.185213,0.100901,0.019575,0.003654
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.532980,0.779806,0.762005,0.411012,0.537889,0.457745,0.332249,0.676727,0.700704,0.738087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0Bs0hUYxz7REyIHH7tRhL2,Helter Skelter - Remastered 2009,The Beatles,When I get to the bottom I go back to the top ...,0.679781,0.456929,0.408230,0.654816,0.505195,0.614792,...,0.511901,0.556455,0.506474,0.440912,0.646144,0.600179,0.531168,0.754828,0.531738,0.973838
491,0bs6tC9MHag57CQU3A0Im1,The Reason (Sean Cook Remix),Urban Rescue,I missed a thousand moments I traded precious ...,0.736059,0.705247,0.085674,0.138247,0.170940,0.135576,...,0.152986,0.313495,0.557643,0.238599,0.484598,0.398980,0.859644,0.324805,0.343791,0.506000
492,0bSr5pybJSlgG5OZH441gF,Simply Irresistible,Robert Palmer,How can it be permissible? She compromise my p...,0.756023,0.681747,0.638927,0.277598,0.075739,0.066175,...,0.713942,0.739583,0.808448,0.686018,0.846991,0.746461,0.841321,0.807028,0.827779,0.725122
493,0BTAWeXFNhgAYixu5YGtE8,Red,HyunA,"So cool하게, 더 hot하게 Red lipstick 좀 더 빨갛게 (빨갛게) ...",0.720760,0.879770,0.485887,0.595712,0.573221,0.278929,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Limipieza de datos luego del merge:

- **Eliminamos Datos nulos**

- **Eliminamos Datos duplicados**



In [70]:
combined_data = combined_data.dropna() #Elimino la tabla con valores nulos
combined_data = combined_data.drop_duplicates() #Elimino los duplicados

combined_data.shape

(495, 12004)

Ahora realizaremos la normalizacion del vector caracteristico para eso de la dataframe mezclada solo eligiremos columnas pertenecientes al vector caracteristico:
feature1 hasta feature50

In [71]:

# Seleccionar solo las columnas numéricas
features_to_normalize = combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
features_to_normalize.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0.470423,0.732632,0.855973,0.962355,0.603475,0.635956,0.953577,0.934506,0.957968,0.799275,...,0.038902,0.122038,0.073865,0.021143,0.019284,0.063284,0.059732,0.060316,0.026597,0.048140
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.405669,0.820913,0.503202,...,0.551716,0.223094,0.291287,0.160346,0.045806,0.151824,0.163924,0.124293,0.098941,0.066165
2,0.639059,0.712995,0.630074,0.718678,0.465735,0.600425,0.563980,0.436771,0.708230,0.478281,...,0.564603,0.547191,0.585334,0.650047,0.622460,0.714395,0.713101,0.656200,0.686142,0.817051
3,0.690065,0.800364,0.246730,0.239026,0.128097,0.170915,0.207017,0.194962,0.123076,0.188414,...,0.105633,0.068960,0.088050,0.215053,0.203416,0.047231,0.185213,0.100901,0.019575,0.003654
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.532980,0.779806,0.762005,0.411012,0.537889,0.457745,0.332249,0.676727,0.700704,0.738087


Alineamos las filas con el conjunto de datos

In [72]:
features_to_normalize = features_to_normalize.loc[combined_data.index]
features_to_normalize



,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0.470423,0.732632,0.855973,0.962355,0.603475,0.635956,0.953577,0.934506,0.957968,0.799275,...,0.038902,0.122038,0.073865,0.021143,0.019284,0.063284,0.059732,0.060316,0.026597,0.048140
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.405669,0.820913,0.503202,...,0.551716,0.223094,0.291287,0.160346,0.045806,0.151824,0.163924,0.124293,0.098941,0.066165
2,0.639059,0.712995,0.630074,0.718678,0.465735,0.600425,0.563980,0.436771,0.708230,0.478281,...,0.564603,0.547191,0.585334,0.650047,0.622460,0.714395,0.713101,0.656200,0.686142,0.817051
3,0.690065,0.800364,0.246730,0.239026,0.128097,0.170915,0.207017,0.194962,0.123076,0.188414,...,0.105633,0.068960,0.088050,0.215053,0.203416,0.047231,0.185213,0.100901,0.019575,0.003654
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.532980,0.779806,0.762005,0.411012,0.537889,0.457745,0.332249,0.676727,0.700704,0.738087
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0.679781,0.456929,0.408230,0.654816,0.505195,0.614792,0.687347,0.597343,0.430745,0.408250,...,0.511901,0.556455,0.506474,0.440912,0.646144,0.600179,0.531168,0.754828,0.531738,0.973838
491,0.736059,0.705247,0.085674,0.138247,0.170940,0.135576,0.098177,0.149078,0.190791,0.470473,...,0.152986,0.313495,0.557643,0.238599,0.484598,0.398980,0.859644,0.324805,0.343791,0.506000
492,0.756023,0.681747,0.638927,0.277598,0.075739,0.066175,0.031747,0.040969,0.097656,0.029726,...,0.713942,0.739583,0.808448,0.686018,0.846991,0.746461,0.841321,0.807028,0.827779,0.725122
493,0.720760,0.879770,0.485887,0.595712,0.573221,0.278929,0.621039,0.534421,0.667202,0.403254,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


Normalizamos las características numéricas del vector caracteristico con scaler.

In [73]:
scaler = preprocessing.StandardScaler()
scaled_features = scaler.fit_transform(features_to_normalize)
scaled_features

array([[ 0.35606695,  1.19519794,  1.65511529, ..., -1.29718468,
        -1.46262556, -1.34313435],
       [-1.14099644, -1.16551623, -1.20864581, ..., -1.05598115,
        -1.18674987, -1.27608357],
       [ 0.89273039,  1.13192064,  0.89934201, ...,  0.94940362,
         1.0524915 ,  1.51705681],
       ...,
       [ 1.26495144,  1.03123201,  0.92896146, ...,  1.51805267,
         1.59261271,  1.17509922],
       [ 1.15273402,  1.66930892,  0.41694829, ..., -1.52458772,
        -1.564051  , -1.52220334],
       [ 1.22052292, -0.12450232, -1.1812084 , ...,  0.13104056,
        -1.121096  , -1.25568265]])

Combinamos las características normalizadas con las otras columnas en una nueva dataframe con los datos normalizados

In [74]:
normalized_combined_data = pd.concat([combined_data[['track_id', 'track_name', 'track_artist', 'lyrics']], pd.DataFrame(scaled_features, columns=features_to_normalize.columns)], axis=1)
normalized_combined_data.head()


,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0.356067,1.195198,1.655115,2.137754,0.846012,0.924700,...,-1.432195,-1.124998,-1.335076,-1.580172,-1.486907,-1.319937,-1.284728,-1.297185,-1.462626,-1.343134
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.551141,-0.734370,-0.482292,-1.020692,-1.382170,-0.959301,-0.892171,-1.055981,-1.186750,-1.276084
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.892730,1.131921,0.899342,1.290392,0.366481,0.802877,...,0.600981,0.518424,0.671031,0.947483,0.895104,1.332146,1.176915,0.949404,1.052491,1.517057
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,1.055051,1.413445,-0.383180,-0.377554,-0.808973,-0.669751,...,-1.174109,-1.330171,-1.279439,-0.800820,-0.759751,-1.385326,-0.811966,-1.144174,-1.489405,-1.508612
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.478676,1.417593,1.363977,-0.013232,0.561121,0.286766,-0.257989,1.026792,1.108024,1.223329


Limpieza de datos a esta nueva dataframe:

- **Verificar si hay valores NaN en el DataFrame**

- **Mostrar las filas con al menos un valor NaN**

- **Mostrar el número total de filas con al menos un valor NaN**

In [75]:
nan_check = normalized_combined_data.isna().any(axis=1)

rows_with_nan = normalized_combined_data[nan_check]
print(f"Filas con al menos un valor NaN:\n{rows_with_nan}")

total_rows_with_nan = rows_with_nan.shape[0]
print(f"Número total de filas con al menos un valor NaN: {total_rows_with_nan}")


Filas con al menos un valor NaN:
Empty DataFrame
Columns: [track_id, track_name, track_artist, lyrics, f1, f2, f3, f4, f5, f6, f7, f8, f9, f10, f11, f12, f13, f14, f15, f16, f17, f18, f19, f20, f21, f22, f23, f24, f25, f26, f27, f28, f29, f30, f31, f32, f33, f34, f35, f36, f37, f38, f39, f40, f41, f42, f43, f44, f45, f46, f47, f48, f49, f50, f51, f52, f53, f54, f55, f56, f57, f58, f59, f60, f61, f62, f63, f64, f65, f66, f67, f68, f69, f70, f71, f72, f73, f74, f75, f76, f77, f78, f79, f80, f81, f82, f83, f84, f85, f86, f87, f88, f89, f90, f91, f92, f93, f94, f95, f96, ...]
Index: []

[0 rows x 12004 columns]
Número total de filas con al menos un valor NaN: 0


Verficamos nuestra dataframe limpia.

In [76]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0.356067,1.195198,1.655115,2.137754,0.846012,0.924700,...,-1.432195,-1.124998,-1.335076,-1.580172,-1.486907,-1.319937,-1.284728,-1.297185,-1.462626,-1.343134
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.551141,-0.734370,-0.482292,-1.020692,-1.382170,-0.959301,-0.892171,-1.055981,-1.186750,-1.276084
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.892730,1.131921,0.899342,1.290392,0.366481,0.802877,...,0.600981,0.518424,0.671031,0.947483,0.895104,1.332146,1.176915,0.949404,1.052491,1.517057
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,1.055051,1.413445,-0.383180,-0.377554,-0.808973,-0.669751,...,-1.174109,-1.330171,-1.279439,-0.800820,-0.759751,-1.385326,-0.811966,-1.144174,-1.489405,-1.508612
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.478676,1.417593,1.363977,-0.013232,0.561121,0.286766,-0.257989,1.026792,1.108024,1.223329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0Bs0hUYxz7REyIHH7tRhL2,Helter Skelter - Remastered 2009,The Beatles,When I get to the bottom I go back to the top ...,1.022321,0.306816,0.157138,1.068316,0.503859,0.852138,...,0.397152,0.554236,0.361723,0.106942,0.988634,0.866926,0.491461,1.321248,0.463686,2.100272
491,0bs6tC9MHag57CQU3A0Im1,The Reason (Sean Cook Remix),Urban Rescue,I missed a thousand moments I traded precious ...,1.201419,1.106955,-0.922012,-0.728003,-0.659820,-0.790915,...,-0.990971,-0.384923,0.562419,-0.706184,0.350670,0.047408,1.729033,-0.300014,-0.253035,0.360014
492,0bSr5pybJSlgG5OZH441gF,Simply Irresistible,Robert Palmer,How can it be permissible? She compromise my p...,1.264951,1.031232,0.928961,-0.243423,-0.991251,-1.028866,...,1.178558,1.262111,1.546138,1.092059,1.781801,1.462756,1.659999,1.518053,1.592613,1.175099
493,0BTAWeXFNhgAYixu5YGtE8,Red,HyunA,"So cool하게, 더 hot하게 Red lipstick 좀 더 빨갛게 (빨갛게) ...",1.152734,1.669309,0.416948,0.862790,0.740685,-0.299411,...,-1.582651,-1.596735,-1.624793,-1.665146,-1.563064,-1.577705,-1.509776,-1.524588,-1.564051,-1.522203


Esta parte parece ser innecesaria, pero se realiza y es que a partir del vector caracteristico se normalizara ahora con la columna lyrycs de la cancion para obetener un vector caracteristico mas exacto.

In [77]:
# Seleccionar las características numéricas normalizadas
numeric_features = normalized_combined_data.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
numeric_features

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0.356067,1.195198,1.655115,2.137754,0.846012,0.924700,1.894512,1.941987,2.028300,1.507384,...,-1.432195,-1.124998,-1.335076,-1.580172,-1.486907,-1.319937,-1.284728,-1.297185,-1.462626,-1.343134
1,-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,-1.253127,0.148866,1.558453,0.485577,...,0.551141,-0.734370,-0.482292,-1.020692,-1.382170,-0.959301,-0.892171,-1.055981,-1.186750,-1.276084
2,0.892730,1.131921,0.899342,1.290392,0.366481,0.802877,0.608502,0.254324,1.172157,0.399569,...,0.600981,0.518424,0.671031,0.947483,0.895104,1.332146,1.176915,0.949404,1.052491,1.517057
3,1.055051,1.413445,-0.383180,-0.377554,-0.808973,-0.669751,-0.569791,-0.565574,-0.833848,-0.600820,...,-1.174109,-1.330171,-1.279439,-0.800820,-0.759751,-1.385326,-0.811966,-1.144174,-1.489405,-1.508612
4,-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,-1.253127,-1.226629,-1.255774,-1.251074,...,0.478676,1.417593,1.363977,-0.013232,0.561121,0.286766,-0.257989,1.026792,1.108024,1.223329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,1.022321,0.306816,0.157138,1.068316,0.503859,0.852138,1.015720,0.798775,0.220892,0.157879,...,0.397152,0.554236,0.361723,0.106942,0.988634,0.866926,0.491461,1.321248,0.463686,2.100272
491,1.201419,1.106955,-0.922012,-0.728003,-0.659820,-0.790915,-0.929059,-0.721153,-0.601710,0.372621,...,-0.990971,-0.384923,0.562419,-0.706184,0.350670,0.047408,1.729033,-0.300014,-0.253035,0.360014
492,1.264951,1.031232,0.928961,-0.243423,-0.991251,-1.028866,-1.148336,-1.087715,-0.920992,-1.148484,...,1.178558,1.262111,1.546138,1.092059,1.781801,1.462756,1.659999,1.518053,1.592613,1.175099
493,1.152734,1.669309,0.416948,0.862790,0.740685,-0.299411,0.796845,0.585426,1.031507,0.140635,...,-1.582651,-1.596735,-1.624793,-1.665146,-1.563064,-1.577705,-1.509776,-1.524588,-1.564051,-1.522203


In [78]:
# Crear un nuevo vectorizador para las letras
text_vectorizer = TfidfVectorizer()
text_features = text_vectorizer.fit_transform(normalized_combined_data["lyrics"])
text_features

<495x13567 sparse matrix of type '<class 'numpy.float64'>'
	with 58261 stored elements in Compressed Sparse Row format>

In [79]:
scaler = preprocessing.StandardScaler()
scaled_numeric_features = scaler.fit_transform(numeric_features)
scaled_numeric_features

array([[ 0.35606695,  1.19519794,  1.65511529, ..., -1.29718468,
        -1.46262556, -1.34313435],
       [-1.14099644, -1.16551623, -1.20864581, ..., -1.05598115,
        -1.18674987, -1.27608357],
       [ 0.89273039,  1.13192064,  0.89934201, ...,  0.94940362,
         1.0524915 ,  1.51705681],
       ...,
       [ 1.26495144,  1.03123201,  0.92896146, ...,  1.51805267,
         1.59261271,  1.17509922],
       [ 1.15273402,  1.66930892,  0.41694829, ..., -1.52458772,
        -1.564051  , -1.52220334],
       [ 1.22052292, -0.12450232, -1.1812084 , ...,  0.13104056,
        -1.121096  , -1.25568265]])

In [80]:
# Combinar las características normalizadas con las otras columnas
combined_features = np.hstack((scaled_numeric_features, text_features.toarray()))
combined_features

array([[ 0.35606695,  1.19519794,  1.65511529, ...,  0.        ,
         0.        ,  0.        ],
       [-1.14099644, -1.16551623, -1.20864581, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.89273039,  1.13192064,  0.89934201, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.26495144,  1.03123201,  0.92896146, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.15273402,  1.66930892,  0.41694829, ...,  0.29364195,
         0.0293642 ,  0.        ],
       [ 1.22052292, -0.12450232, -1.1812084 , ...,  0.        ,
         0.        ,  0.        ]])

Obtenemos el mismo vector caracteristico no hay diferencia.

In [81]:
normalized_combined_data

,track_id,track_name,track_artist,lyrics,f1,f2,f3,f4,f5,f6,...,f11991,f11992,f11993,f11994,f11995,f11996,f11997,f11998,f11999,f12000
0,0017A6SJgTbfQVU2EtsPNo,Pangarap,Barbie's Cradle,Minsan pa Nang ako'y napalingon Hindi ko alam ...,0.356067,1.195198,1.655115,2.137754,0.846012,0.924700,...,-1.432195,-1.124998,-1.335076,-1.580172,-1.486907,-1.319937,-1.284728,-1.297185,-1.462626,-1.343134
1,004s3t0ONYlzxII9PLgU6z,I Feel Alive,Steady Rollin,"The trees, are singing in the wind The sky blu...",-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.551141,-0.734370,-0.482292,-1.020692,-1.382170,-0.959301,-0.892171,-1.055981,-1.186750,-1.276084
2,00chLpzhgVjxs1zKC9UScL,Poison,Bell Biv DeVoe,"NA Yeah, Spyderman and Freeze in full effect U...",0.892730,1.131921,0.899342,1.290392,0.366481,0.802877,...,0.600981,0.518424,0.671031,0.947483,0.895104,1.332146,1.176915,0.949404,1.052491,1.517057
3,00cqd6ZsSkLZqGMlQCR0Zo,Baby It's Cold Outside (feat. Christina Aguilera),CeeLo Green,I really can't stay Baby it's cold outside I'v...,1.055051,1.413445,-0.383180,-0.377554,-0.808973,-0.669751,...,-1.174109,-1.330171,-1.279439,-0.800820,-0.759751,-1.385326,-0.811966,-1.144174,-1.489405,-1.508612
4,00emjlCv9azBN0fzuuyLqy,Dumb Litty,KARD,Get up out of my business You don't keep me fr...,-1.140996,-1.165516,-1.208646,-1.208743,-1.254930,-1.255753,...,0.478676,1.417593,1.363977,-0.013232,0.561121,0.286766,-0.257989,1.026792,1.108024,1.223329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,0Bs0hUYxz7REyIHH7tRhL2,Helter Skelter - Remastered 2009,The Beatles,When I get to the bottom I go back to the top ...,1.022321,0.306816,0.157138,1.068316,0.503859,0.852138,...,0.397152,0.554236,0.361723,0.106942,0.988634,0.866926,0.491461,1.321248,0.463686,2.100272
491,0bs6tC9MHag57CQU3A0Im1,The Reason (Sean Cook Remix),Urban Rescue,I missed a thousand moments I traded precious ...,1.201419,1.106955,-0.922012,-0.728003,-0.659820,-0.790915,...,-0.990971,-0.384923,0.562419,-0.706184,0.350670,0.047408,1.729033,-0.300014,-0.253035,0.360014
492,0bSr5pybJSlgG5OZH441gF,Simply Irresistible,Robert Palmer,How can it be permissible? She compromise my p...,1.264951,1.031232,0.928961,-0.243423,-0.991251,-1.028866,...,1.178558,1.262111,1.546138,1.092059,1.781801,1.462756,1.659999,1.518053,1.592613,1.175099
493,0BTAWeXFNhgAYixu5YGtE8,Red,HyunA,"So cool하게, 더 hot하게 Red lipstick 좀 더 빨갛게 (빨갛게) ...",1.152734,1.669309,0.416948,0.862790,0.740685,-0.299411,...,-1.582651,-1.596735,-1.624793,-1.665146,-1.563064,-1.577705,-1.509776,-1.524588,-1.564051,-1.522203


Esta parte es necesaria para la query de modo que crearemos un vector caracteristico para esta consulta, necesariamente la misma cantidad de caracteristicas que de las canciones.

In [82]:
def extract_and_normalize_features(audio_path, normalized_combined_data):
    y, sr = librosa.load(audio_path)

    # Extraer características de audio (aquí puedes elegir las características que deseas)
    features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=12000)  # Ahora, extraer 12000 coeficientes MFCC

    # Asegurarse de que las características tengan el mismo tamaño que las del DataFrame normalized_combined_data
    max_length = normalized_combined_data.shape[1] - 4

    # Ajustar el tamaño de las características (rellenando con ceros si es necesario)
    if features.shape[1] < max_length:
        pad_width = max_length - features.shape[1]
        features = np.pad(features, pad_width=((0, 0), (0, pad_width)))

    # Crear un DataFrame con las características del audio
    audio_features = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(1, 12001)])

    # Convertir el DataFrame de características de audio a un array NumPy
    audio_features_array = audio_features.values

    # Normalizar el vector de características de audio
    scaled_audio_features = scaler.transform(audio_features_array)

    # Convertir el array NumPy a un DataFrame
    scaled_audio_features = pd.DataFrame(scaled_audio_features, columns=[f'feature_{i}' for i in range(1, 12001)])

    return scaled_audio_features


Consulta 2

In [83]:
audio_path="../query2.wav"
scaled_audio_features2=extract_and_normalize_features(audio_path, normalized_combined_data)
scaled_audio_features2

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_11991,feature_11992,feature_11993,feature_11994,feature_11995,feature_11996,feature_11997,feature_11998,feature_11999,feature_12000
0,-435.298279,-432.778809,-491.277191,-527.434998,-526.483032,-522.620422,-521.851257,-522.604980,-518.619934,-514.677246,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
1,61.816315,63.625111,27.193474,4.778519,4.786145,8.438463,10.664425,8.868388,7.793067,8.470732,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
2,-55.187035,-55.401085,-27.905628,-4.767743,0.650598,-1.385258,-1.125632,-1.032184,-3.864204,-4.688291,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
3,-52.830360,-52.119423,-28.434399,-6.015786,-4.101000,-3.008466,-1.221179,-0.475851,0.582900,0.213655,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
4,10.062284,12.080715,7.896958,1.653944,-2.521240,0.098991,2.873945,0.940562,-0.879641,-1.985665,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-1.098422,-0.702476,-0.584366,-0.297125,0.215557,0.682936,1.958951,1.384546,0.835650,2.318981,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
124,-1.959123,-1.521884,0.315836,0.297115,1.229116,0.066248,0.536730,0.264844,0.020233,-1.256701,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
125,0.172777,1.210672,0.571567,0.673255,0.338192,-0.625111,-0.913269,-0.129975,0.376198,-1.778493,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
126,0.954302,1.602627,-0.973606,-0.442813,0.959123,-0.101996,0.427108,0.916409,2.005571,0.559360,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17


Consulta 1

In [84]:
audio_path="../query.wav"
scaled_audio_features=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_11991,feature_11992,feature_11993,feature_11994,feature_11995,feature_11996,feature_11997,feature_11998,feature_11999,feature_12000
0,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,-5.688046e+02,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
1,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
2,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
3,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
4,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
124,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
125,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
126,-5.382900e-17,-7.715489e-17,1.435440e-16,3.050310e-17,6.459479e-17,-4.665180e-17,1.650756e-16,-1.166295e-17,1.578984e-16,-2.601735e-16,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17


Consulta 3

In [85]:
audio_path="../query3.wav"
scaled_audio_features3=extract_and_normalize_features(audio_path, normalized_combined_data)

scaled_audio_features3

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,...,feature_11991,feature_11992,feature_11993,feature_11994,feature_11995,feature_11996,feature_11997,feature_11998,feature_11999,feature_12000
0,-452.724274,-410.642242,-382.672241,-369.403137,-319.774872,-283.140106,-261.333923,-245.908600,-241.301178,-234.242737,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
1,16.924856,37.878498,53.071121,54.236336,49.737312,47.321243,46.468109,50.987701,49.823235,45.855473,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
2,2.595886,-12.398136,-20.443975,-33.565346,-72.241028,-86.934692,-90.179291,-91.479698,-90.874260,-83.739441,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
3,0.188916,7.635675,9.030219,5.604999,18.592773,29.668083,39.156750,45.619164,54.013000,61.148346,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
4,9.110559,25.848841,28.894205,29.356882,38.982628,33.725739,24.050903,20.416084,21.267445,16.894958,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,1.014865,0.491443,0.105006,-1.209732,-2.149136,-1.459698,-1.026202,-0.505115,-2.157951,-2.793029,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
124,-1.337445,0.952938,1.291588,0.369337,-0.984945,0.211321,0.724087,-0.481273,-2.745622,-2.903449,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
125,-0.919728,-2.540533,-2.342331,-0.719841,1.260162,2.068654,0.259312,-0.320774,-1.088140,-1.899452,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17
126,0.900744,-3.300434,-3.379250,-2.151307,-0.587406,1.479919,0.253590,1.374237,1.542091,0.965557,...,-7.177200e-18,-6.459479e-17,2.153160e-17,1.435440e-17,1.076580e-16,-8.612639e-17,-1.256010e-16,-7.177199e-17,-4.306319e-17,3.588600e-17


EL modelo KNN search se implementa con ayudade sklearn que nos proporciona la facilidad con los modelos de dos arrays se logra obtener los k vecinos mas cercanos

# Sin cola de prioridad

In [86]:
from sklearn.neighbors import NearestNeighbors

def knn_search(query_audio_features, reference_features, k=5, metric='cosine'):
    # Inicializar el modelo KNN
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extraer las características numéricas para el modelo KNN, 4 primeras columnas del data frame
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convertir query_audio_features a un array NumPy
    query_features_numeric = query_audio_features.to_numpy()

    # Utilizar el modelo para encontrar los k vecinos más cercanos para la query del audio
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Obtener las k canciones más cercanas utilizando los índices
    nearest_songs = reference_features.iloc[indices[0], :4]  # Obtener solo las primeras 4 columnas

    return nearest_songs


# Con cola de prioridad

In [87]:
from queue import PriorityQueue
from sklearn.neighbors import NearestNeighbors

def knn_search_priority_queue(query_audio_features, reference_features, k=5, metric='cosine'):
    # Initialize the KNN model
    knn_model = NearestNeighbors(n_neighbors=k, metric=metric)

    # Extract the numeric features for the KNN model, excluding the first 4 columns
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)
    knn_model.fit(reference_features_numeric)

    # Convert query_audio_features to a NumPy array
    query_features_numeric = query_audio_features.to_numpy()

    # Use the model to find the k nearest neighbors for the audio query
    distances, indices = knn_model.kneighbors(query_features_numeric)

    # Create a priority queue to store the k nearest neighbors
    pq = PriorityQueue()

    # Iterate over the indices and distances to add the neighbors to the priority queue
    for i in range(len(indices[0])):
        index = indices[0][i]
        distance = distances[0][i]
        pq.put((distance, index))

    # Get the k nearest neighbors from the priority queue
    nearest_songs = []
    while not pq.empty():
        distance, index = pq.get()
        nearest_songs.append(reference_features.iloc[index, :4])  # Get only the first 4 columns

    return nearest_songs


## Query 1


In [88]:
k_nearest_songs = knn_search(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
231,06WD650Zt5MQ55kFMP2f3r,Fire Fly,Childish Gambino,Now when they see us on the streets All they w...
429,0B8B8cVRFIG1yznoQe7c9s,Setting Fires,The Chainsmokers,Down to my last match fire I'll touch just to ...
232,06WohfoFEUa5faNy8f9S5N,Adrenalize,In This Moment,Adrenalize me Come a little bit closer Before ...
88,02lvTeAFIIsOd0wVNoRkQ5,Who Needs You - Remastered 2011,Queen,"I make it half past six, you come at seven Alw..."
472,0BMXB8USeC2zVufRWBcMuY,Vete,Khea,"NA Y si querés vete, my girl Si yo ya sé, si t..."


In [89]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
231,06WD650Zt5MQ55kFMP2f3r,Fire Fly,Childish Gambino,Now when they see us on the streets All they w...
429,0B8B8cVRFIG1yznoQe7c9s,Setting Fires,The Chainsmokers,Down to my last match fire I'll touch just to ...
232,06WohfoFEUa5faNy8f9S5N,Adrenalize,In This Moment,Adrenalize me Come a little bit closer Before ...
88,02lvTeAFIIsOd0wVNoRkQ5,Who Needs You - Remastered 2011,Queen,"I make it half past six, you come at seven Alw..."
472,0BMXB8USeC2zVufRWBcMuY,Vete,Khea,"NA Y si querés vete, my girl Si yo ya sé, si t..."


## Query 2

In [90]:
k_nearest_songs = knn_search(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
494,0BTGqPIW9acmhhUmENkq5r,If You Love Me,Brownstone,I don't wanna rain on this parade But I'm star...
447,0BfLvshnvJldGROufmYCrv,Song For You,Rhye,I saw your tear fall from your grace I fell in...
81,02H05NlKnwFQREf0qu89AL,Compound,Trellion,Bubblegum ice pop (what else?) Pinky ring danc...
148,04JmxH2IpgpIeZdbYswRq1,Azzlack Harami,Milonair,"Azzlack-Harami Hamad 45, Milonair, wir machen ..."
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying..."


In [91]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features2, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
494,0BTGqPIW9acmhhUmENkq5r,If You Love Me,Brownstone,I don't wanna rain on this parade But I'm star...
447,0BfLvshnvJldGROufmYCrv,Song For You,Rhye,I saw your tear fall from your grace I fell in...
81,02H05NlKnwFQREf0qu89AL,Compound,Trellion,Bubblegum ice pop (what else?) Pinky ring danc...
148,04JmxH2IpgpIeZdbYswRq1,Azzlack Harami,Milonair,"Azzlack-Harami Hamad 45, Milonair, wir machen ..."
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying..."


## Query 3

In [92]:
k_nearest_songs = knn_search(scaled_audio_features3, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
494,0BTGqPIW9acmhhUmENkq5r,If You Love Me,Brownstone,I don't wanna rain on this parade But I'm star...
81,02H05NlKnwFQREf0qu89AL,Compound,Trellion,Bubblegum ice pop (what else?) Pinky ring danc...
447,0BfLvshnvJldGROufmYCrv,Song For You,Rhye,I saw your tear fall from your grace I fell in...
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying..."
462,0Bk4P8WODpjxTzu4ry5PGQ,LUNA,Sick Individuals,(Open your eyes) (Everybody) (Everybody) (Ever...


In [93]:
k_nearest_songs = knn_search_priority_queue(scaled_audio_features3, normalized_combined_data, k=5)
result = pd.DataFrame(k_nearest_songs)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist,lyrics
494,0BTGqPIW9acmhhUmENkq5r,If You Love Me,Brownstone,I don't wanna rain on this parade But I'm star...
81,02H05NlKnwFQREf0qu89AL,Compound,Trellion,Bubblegum ice pop (what else?) Pinky ring danc...
447,0BfLvshnvJldGROufmYCrv,Song For You,Rhye,I saw your tear fall from your grace I fell in...
5,00f9VGHfQhAHMCQ2bSjg3D,Soldier,James TW,"Hold your breath, don't look down, keep trying..."
462,0Bk4P8WODpjxTzu4ry5PGQ,LUNA,Sick Individuals,(Open your eyes) (Everybody) (Everybody) (Ever...


## Range Search
El range search utilizamos el modelo KNN y obtenemos resultados interesantes.

In [94]:
from sklearn.neighbors import RadiusNeighborsClassifier


def range_search(query_features=pd.DataFrame(), reference_features=pd.DataFrame(), radius=int()):
    # Solo trabajaremos con el vector característico de las canciones
    reference_features_numeric = reference_features.drop(['track_id', 'track_name', 'track_artist', 'lyrics'], axis=1)

    # Convertimos el dataframe de la consulta a un array numpy
    query_features_numeric = query_features.to_numpy()

    # Creamos el modelo RadiusNeighborsClassifier
    knn_model = RadiusNeighborsClassifier(radius=radius, metric='euclidean')

    # Entrenamos al modelo con el vector característico de las canciones
    knn_model.fit(reference_features_numeric, reference_features['track_id'])

    # Buscamos vecinos dentro del radio para cada punto de la consulta
    neighbor_indices = knn_model.radius_neighbors(query_features_numeric, return_distance=False)

    # Extraemos información de las canciones encontradas
    song_data = []
    unique_tracks = set()

    for query_index, indices in enumerate(neighbor_indices):
        for neighbor_index in indices:
            track_id = reference_features['track_id'].iloc[neighbor_index]
            if track_id not in unique_tracks:
                unique_tracks.add(track_id)
                song_data.append({
                    'track_id': track_id,
                    'track_name': reference_features['track_name'].iloc[neighbor_index],
                    'track_artist': reference_features['track_artist'].iloc[neighbor_index],
                })

    return song_data

## Query 1
Query 1 normalizada mala diccion y bajo volumen. 

In [95]:
radius = 100
indices_within_radius = range_search(scaled_audio_features, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,05xbEV5M8DAQmhFKKlHaDd,Bring It On,Geto Boys
1,00PLtXXER1XcTRZvs3LioS,Get The Funk Out Ma Face,The Brothers Johnson
2,04Rz403IY7QUqgfrTWX6OU,Da Graveyard,Big L
3,05Sgj1Hx03ZXc57gsV1kfX,It's Goin' Down (feat. Nitti),Yung Joc
4,09EwNbGvUyu7BDEYG0cJro,Can't C Me,2Pac
...,...,...,...
74,09PGyODYYlVsL49N7TR914,Pedro Navaja,Rubén Blades
75,075NNuZU6IxC7Us3L7ylep,Guardia Civil,Reda Rwena
76,07K9jPEoVsqoEopODxiAUs,Living in Oblivion,Anything Box
77,09xGWzgSw8zp5oS1x5ogym,Melaza,Tiro De Gracia


## Query 2
Query 2 diccion aceptable y volumen aceptable

In [96]:
radius = 100
indices_within_radius = range_search(scaled_audio_features2, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,04Rz403IY7QUqgfrTWX6OU,Da Graveyard,Big L
1,05xbEV5M8DAQmhFKKlHaDd,Bring It On,Geto Boys
2,00PLtXXER1XcTRZvs3LioS,Get The Funk Out Ma Face,The Brothers Johnson
3,09EwNbGvUyu7BDEYG0cJro,Can't C Me,2Pac
4,0Bpkq4z0VfjkO7kUME8gks,Forever,Drake
...,...,...,...
73,09xGWzgSw8zp5oS1x5ogym,Melaza,Tiro De Gracia
74,037yW9RzsLze4OmBYmcH4G,Save My Night - BlasterJaxx Remix,Armin van Buuren
75,09PGyODYYlVsL49N7TR914,Pedro Navaja,Rubén Blades
76,075NNuZU6IxC7Us3L7ylep,Guardia Civil,Reda Rwena


## Query 3
La query 3 es mas clara mejor diccion y pronunciacion mejor resultado que la query 1, demostrando que si la consulta es una que no tiene una buena diccion y volumen bajo, obtenemos malos resultados.

In [97]:
radius = 100
indices_within_radius = range_search(scaled_audio_features3, normalized_combined_data, radius)
result = pd.DataFrame(indices_within_radius)
result

/Users/pierre/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but RadiusNeighborsClassifier was fitted with feature names
  warnings.warn(


,track_id,track_name,track_artist
0,00PLtXXER1XcTRZvs3LioS,Get The Funk Out Ma Face,The Brothers Johnson
1,04Rz403IY7QUqgfrTWX6OU,Da Graveyard,Big L
2,05xbEV5M8DAQmhFKKlHaDd,Bring It On,Geto Boys
3,09EwNbGvUyu7BDEYG0cJro,Can't C Me,2Pac
4,0Bpkq4z0VfjkO7kUME8gks,Forever,Drake
...,...,...,...
73,037yW9RzsLze4OmBYmcH4G,Save My Night - BlasterJaxx Remix,Armin van Buuren
74,09Oj9xexrJ5N5hzMoOHgh5,This Is The Shack,Warren G
75,02mJtllR8CoskckLlrirNV,Smile - Years Remix,Galantis
76,07K9jPEoVsqoEopODxiAUs,Living in Oblivion,Anything Box


## Aclaraciones

La data frame tiene una cancion pero de distintas versiones y tiempo, cuenta con detalles que otros no tienen.

In [98]:
# Filtrar el DataFrame original para obtener todas las instancias de la canción con track_id específico
all_instances = songs[songs['track_id'] == '05xbEV5M8DAQmhFKKlHaDd']

# Imprimir o mostrar el DataFrame con todas las instancias
print(all_instances)


                   track_id   track_name track_artist  \
204  05xbEV5M8DAQmhFKKlHaDd  Bring It On    Geto Boys   

                                                lyrics  track_popularity  \
204  Cover your dome or feel a motherfucking Glock ...                31   

             track_album_id      track_album_name track_album_release_date  \
204  0RKFBA5pwzXGBvcbOkd1g3  Til Death Do Us Part               1993-03-09   

             playlist_name             playlist_id  ... loudness mode  \
204  90's Southern Hip Hop  2NMW1nwQYSVlXd26uLenX6  ...     -9.0    1   

     speechiness  acousticness  instrumentalness  liveness  valence   tempo  \
204        0.331         0.175               0.0     0.651    0.533  92.888   

     duration_ms  language  
204       496133        en  

[1 rows x 25 columns]


In [99]:
# Suponiendo que "track_id" es el nombre de la columna que contiene los identificadores de las canciones
track_id_count = features['track_id'].value_counts().get('05xbEV5M8DAQmhFKKlHaDd', 0)

print(f"El track_id '05xbEV5M8DAQmhFKKlHaDd' aparece {track_id_count} veces en el DataFrame features.")


El track_id '05xbEV5M8DAQmhFKKlHaDd' aparece 1 veces en el DataFrame features.
